# Example: Optimal adversaries for dense MNIST model


## Building and training the neural network

In [1]:
#Import requisite packages
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

Show how to load the dataset for training

In [2]:
#Set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#Build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, transform=transforms.ToTensor())
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Define model

In [3]:
hidden_size = 50

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1  = nn.Linear(784, hidden_size)
        self.hidden2  = nn.Linear(hidden_size, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)      
        return x

Define train and test functions

In [4]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28*28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0; correct = 0
    criterion = nn.NLLLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 28*28))
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Train model on dataset

In [5]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.306386
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.291233
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.543504
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.127922
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.217355

Test set: Average loss: 0.1607, Accuracy: 9482/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.148238
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.214166
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.117340
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.038596
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.105932

Test set: Average loss: 0.1298, Accuracy: 9606/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.180933
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.139834
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.052933
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.034447
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.068873

Test set: Average loss: 0.1127, Accuracy: 9659/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.084686
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Building the MIP formulation

Need to export to ONNX, the PyTorch ONNX exporter needs to write to a file so we generate a temporary file.

In [6]:
import torch.onnx
import tempfile
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

We also define bounds on variables

In [7]:
problem_index = 0
image = dataset2[problem_index][0].view(-1,28*28).detach().numpy()
label = dataset2[problem_index][1]

In [8]:
epsilon_infty = 1e-2
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)
input_bounds = [(float(l), float(u)) for l, u in zip(lb[0], ub[0])]

PyTorch needs to trace the model execution to export it, so we defined a dummy input tensor.

In [9]:
x = dataset2[problem_index][0].view(-1,28*28)

Now we can write the ONNX model and load it back.

In [10]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    torch.onnx.export(
        model,
        x,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load back
    network_definition = load_onnx_neural_network_with_bounds(f.name)

Create Pyomo model

In [11]:
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import NeuralNetworkFormulation

OMLT doesn't include a formulation for sigmoid, so define it here

In [18]:
formulation = NeuralNetworkFormulation(network_definition)
    #activation_constraints={'relu': relu_activation}
#)

m = pyo.ConcreteModel()

m.nn = OmltBlock()
m.nn.build_formulation(formulation)

TypeError: Can't instantiate abstract class NeuralNetworkFormulation with abstract methods input_indexes, output_indexes

In [39]:
m.pprint()

1 Block Declarations
    nn : Size=1, Index=None, Active=True
        5 Set Declarations
            input_assignment_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}
            inputs_set : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}
            layers : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    4 : {140615613157728, 140615614166112, 140615614167216, 140615613157440}
            output_assignment_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            outputs_set : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : 